In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
list_of_df = []
for file in glob.glob("../data/vivino/"+ "/*csv"):
    df = pd.read_csv(file, index_col=None, header=0)
    df["wine_type"] = file.replace("../data/vivino/", "").replace(".csv", "")
    list_of_df.append(df)

frame = pd.concat(list_of_df, axis=0, ignore_index=True)
frame.to_csv("../graphics/graphics_data/vivino.csv")

In [3]:
from re import sub

def snake_case(s):
  return '_'.join(
    sub('([A-Z][a-z]+)', r' \1',
    sub('([A-Z]+)', r' \1',
    s.replace('-', ' ').replace("'", ""))).split()).lower()

frame.columns = [snake_case(x) for x in frame.columns]
frame["year"] = pd.to_numeric(frame.year.apply(lambda x: x.replace("N.V.", "2020")))

In [4]:
frame.rename(columns={"rating":"quality"}, inplace=True)
# frame = frame.drop("winery", axis=1)
frame = frame.drop("name", axis=1)
frame["quality"] = frame.quality * 2
frame = frame.drop("country", axis=1)

In [5]:
df = pd.get_dummies(frame)

In [6]:
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
df_train, df_test = model_selection.train_test_split(
        df, test_size=0.2, random_state=66
)
cols_to_adjust = ["number_of_ratings", "price", "year"] 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PowerTransformer

df_transform = ColumnTransformer(
    [("yao", PowerTransformer(), cols_to_adjust)],
    remainder="passthrough",
)

df_train = pd.DataFrame(df_transform.fit_transform(df_train), columns=df.columns)

X_train = df_train.drop("quality", axis=1)
y_train = df_train["quality"]

/Users/josephdespres/my_env/lib/python3.9/site-packages/numpy/core/_methods.py:232: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/Users/josephdespres/my_env/lib/python3.9/site-packages/numpy/core/_methods.py:243: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


In [7]:
df_test = pd.DataFrame(df_transform.fit_transform(df_test), columns=df.columns)
X_test = df_test.drop("quality", axis=1)
y_test = df_test["quality"]

/Users/josephdespres/my_env/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:3216: RuntimeWarning: overflow encountered in power
  out[pos] = (np.power(x[pos] + 1, lmbda) - 1) / lmbda
/Users/josephdespres/my_env/lib/python3.9/site-packages/numpy/core/_methods.py:232: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/Users/josephdespres/my_env/lib/python3.9/site-packages/numpy/core/_methods.py:243: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


In [8]:
rmse_list = []

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn import svm

regression = LinearRegression()
from sklearn import metrics
yhat = regression.fit(X_train, y_train).predict(X_test)
rmse = metrics.mean_squared_error(yhat, y_test, squared=False)

rmse_list.append(rmse)

In [10]:

import multiprocessing
cores = multiprocessing.cpu_count()

xgboost = xgb.XGBRegressor()

xgboost.fit(X_train, y_train)
yhat = xgboost.predict(X_test)
rmse = metrics.mean_squared_error(yhat, y_test, squared=False)
rmse_list.append(rmse)

In [11]:
from sklearn import svm


In [12]:

support_vector_machine = svm.SVR()
support_vector_machine.fit(X_train, y_train)
yhat = support_vector_machine.predict(X_test)
rmse = metrics.mean_squared_error(yhat, y_test, squared=False)
rmse_list.append(rmse)

In [13]:
models = []
models.append(("xgb", xgboost))
models.append(("logistic_reg", regression))
models.append(("svm", support_vector_machine))

In [14]:
preds = []
for i, model in enumerate(models):
    model = model[1]
    yhat = model.fit(X_train, y_train).predict(X_test)
    preds.append(yhat)


In [15]:
yhat = np.array(preds).mean(axis=0)
rmse = metrics.mean_squared_error(yhat, y_test, squared=False)
rmse_list.append(rmse)

In [16]:
rmse_list

[1476143901.0359075, 0.9127213276706317, 0.8793859458985559, 492047967.0128748]

In [17]:
data = pd.DataFrame([rmse_list], columns = ["XGBoost", "Elastic Net", "SVM", "Ensemble"])

In [18]:
# data.to_csv("../graphics/graphics_data/accuracy_table.csv", mode='a', index=True, header=False) 